# Decorators in Python


## Tuple and Dictionary packing und unpacking

In Python können tuple wie `(1, 2, 3)` mit dem `*` symbol "entpackt" werden.

In [1]:
t = (1, 2, 3)
print(t)
print(*t)

(1, 2, 3)
1 2 3


Das Gleiche gilt für Dictionarys.

Um dictionarys zu entpacken müssen die keys zu den Keyword Argumenten der Funtion passen.

In [25]:
print(d)
print("-"*20)

d = {
    'sep': ', ',
    'end': '\ndone\n'
}
print(*t, **d)

{'sep': ', ', 'end': '\ndone\n'}
--------------------
1, 2, 3
done


Der `*`-Operator wird auch in der Funktionsdefinition verwendet, um eine beliebige Anzahl von Argumenten zu akzeptieren.

In [3]:
def f(*args):
    print(args)
    
f(1, 2, 3)

(1, 2, 3)


Wir können auch mit `**` arbeiten, um eine beliebige Anzahl von Schlüssel-Wert-Paaren zu übergeben:

In [4]:
def g(**kwargs):
    print(kwargs)
    
g(a=1, b=2, c=3)

{'a': 1, 'b': 2, 'c': 3}


Mit `*` werden die beliebig viele positional arguments als Tuple zusammengefasst. Mit `**` werden die beliebig viele keyword arguments als Dictionary zusammengefasst.

Diese Technik erlaubt es uns Funktionen zu schreiben, die eine beliebige Anzahl von Argumenten akzeptieren.

In [5]:
def h(*args, **kwargs):
    print(args)
    print(kwargs)
    
h(1, 2, 3, a=1, b=2, c=3)

(1, 2, 3)
{'a': 1, 'b': 2, 'c': 3}


## Higher order functions

In Python ist alles ein Objekt, einschließlich Funktionen. Funktionen werden daher auch als *first class citizens* bezeichnet. 

Das bedeutet Folgendes:

1. **Funktionen als Argumente**: Sie können Funktionen als Argumente an andere Funktionen übergeben.
2. **Funktionen als Rückgabewerte**: Funktionen können auch als Rückgabewerte von anderen Funktionen genutzt werden.

Eine Funktion, die eine andere Funktion als Argument erhält, wird als *higher order function* (höherwertige Funktion) bezeichnet.

In [6]:
def add_5(x):
    return x + 5

def apply(func, x):
    return func(x)

print(apply(add_5, 10))

15


In [7]:
def create_add_n(n):
    def add_n(x):
        return x + n
    return add_n

add_5 = create_add_n(5)

print(add_5(10))

15


Diese Eigenschaft von Python können wir nutzen, um Funktionen im Nachhinein mit weiterer Funktionalität zu erweitern.

In [8]:
def is_prime(n):
    '''A simple function to check if a number is prime.'''
    if n <= 1:
        return False
    return not any(n % i == 0 for i in range(2, n))

def log_function(func):
    def wrapper(n):
        print(f'Calling func with parameter {n}')
        return func(n)
    return wrapper


In [9]:
logged_is_prime = log_function(is_prime)

# call pure function
print(is_prime(17))

print("-"*20)
# call logged function
print(logged_is_prime(17))
    

True
--------------------
Calling func with parameter 17
True


Unser Decorator funktioniert gerade nur dann, wenn die zu dekorierende Funktion genau ein Argument erwartet.
Wir können das Problem lösen, indem wir *args und **kwargs in der inneren Funktion verwenden.
Das bedeutet, dass die dekorierte Funktion beliebig viele Argumente akzeptieren kann. Hier ist der aktualisierte Code:


In [10]:
def log_function(func):
    def wrapper(*args, **kwargs):
        print(f'Calling func with parameter {args} and {kwargs}')
        return func(*args, **kwargs)
    return wrapper

In [11]:
is_prime_logged = log_function(is_prime)

print(is_prime_logged(17))

Calling func with parameter (17,) and {}
True


In [12]:
def filter_primes(*numbers):
    return [n for n in numbers if is_prime(n)]

print(filter_primes(4, 6, 9, 17, 18, 19, 22, 23))

print("-"*20)	    

filter_primes_logged = log_function(filter_primes)

print(filter_primes_logged(4, 6, 9, 17, 18, 19, 22, 23))

[17, 19, 23]
--------------------
Calling func with parameter (4, 6, 9, 17, 18, 19, 22, 23) and {}
[17, 19, 23]


Python bietet uns eine weitere Möglichkeit an um Funktionen direkt bei der Definition zu dekorieren.
Hierfür wird das @-Zeichen verwendet. Anstatt
```python
decorated_function = decorator_function(original_function)
```

Zu schreiben können wir auch einfach
```python
@decorator_function
def original_function():
    pass
```
Verwenden.

Hier nochmal beide Funktionen zusammen:

In [13]:
def log_function(func):
    def wrapper(*args, **kwargs):
        print(f'Calling func with parameter {args} and {kwargs}')
        result = func(*args, **kwargs)
        return result
    return wrapper

@log_function
def is_prime(n):
    '''A simple function to check if a number is prime.'''
    if n <= 1:
        return False
    return not any(n % i == 0 for i in range(2, n))

In [14]:
print(is_prime(17))

Calling func with parameter (17,) and {}
True


Da unsere Funktion durch diese syntax überschrieben wird ergeibt sich ein Problem:
Möchten wir eigenschaften der Funktion abfrage, wie den Namen oder den Docstring, werden uns nun die Eigenschaften unserer wrapper Funktion zurückgegeben.

In [15]:
print(is_prime.__name__)
print(is_prime.__doc__)

wrapper
None


Um dieses Problem zu beheben, können wir direkt einen decorator aus dem functools Modul aus der Standard Bibliothek von Python benutzen.

In [16]:
from functools import wraps

def log_function(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f'Calling func with parameter {args} and {kwargs}')
        result = func(*args, **kwargs)
        return result
    return wrapper

@log_function
def is_prime(n):
    '''A simple function to check if a number is prime.'''
    if n <= 1:
        return False
    return not any(n % i == 0 for i in range(2, n))

In [17]:
print(is_prime.__name__)
print(is_prime.__doc__)

is_prime
A simple function to check if a number is prime.


# Classmethod und Staticmethod

In [18]:
class A:
    
    def m1():
        print("m1 of A called")
        
    def m2(self):
        print("m2 of A called with self", self)
        
    @staticmethod
    def m3():
        print("m3 of A called")
        
    @classmethod
    def m4(cls):
        print("m4 of A called with cls", cls)
        
        
a = A()


# a.m1()
a.m2()
a.m3()
a.m4()

print("-"*20)

A.m1()
A.m2(a)
A.m3()
A.m4()

m2 of A called with self <__main__.A object at 0x000002A51D4D34A0>
m3 of A called
m4 of A called with cls <class '__main__.A'>
--------------------
m1 of A called
m2 of A called with self <__main__.A object at 0x000002A51D4D34A0>
m3 of A called
m4 of A called with cls <class '__main__.A'>


# Property decorator

In [19]:
# demonstrate the property decorator

class B():
    
    def __init__(self, x):
        self.x = x
        
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        if value < 0:
            raise ValueError("x must be positive")
        self._x = value
        
        
b = B(5)
print(b.x)
b.x = 10
print(b.x)

b.x = -1



5
10


ValueError: x must be positive

In [20]:
from datetime import date

class Person:
    def __init__(self, birth_date, death_date=None):
        self._birth_date = birth_date
        self._death_date = death_date

    @property
    def birth_date(self):
        """Get the birth date of the person."""
        return self._birth_date

    @property
    def death_date(self):
        """Get the death date of the person, if applicable."""
        return self._death_date

    @death_date.setter
    def death_date(self, value):
        """Set the death date of the person."""
        if value < self._birth_date:
            raise ValueError("Death date cannot be before birth date")
        self._death_date = value

    @property
    def age(self):
        """Calculate the age of the person based on birth and death date."""
        end_date = self._death_date if self._death_date else date.today()
        return end_date.year - self._birth_date.year - (
            (end_date.month, end_date.day) < (self._birth_date.month, self._birth_date.day)
        )

# Example usage
person = Person(date(1990, 5, 15))
print(f"Birth Date: {person.birth_date}")
print(f"Current Age: {person.age}")

# Set a death date
person.death_date = date(2020, 5, 14)
print(f"Death Date: {person.death_date}")
print(f"Age at Death: {person.age}")

person.death_date = date(1980, 5, 14)


Birth Date: 1990-05-15
Current Age: 34
Death Date: 2020-05-14
Age at Death: 29


ValueError: Death date cannot be before birth date

# lru_cache

In [21]:
from functools import lru_cache

# Define a Fibonacci function with caching
@lru_cache(maxsize=None)  # maxsize=None means unlimited cache size
def fibonacci(n):
    """Return the nth Fibonacci number."""
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

# Example usage
print(fibonacci(10))  # Output: 55
print(fibonacci(20))  # Output: 6765

# Calling fibonacci(10) again will use the cached result
print(fibonacci(10))  # Output: 55 (retrieved from cache)


55
6765
55


# Data Class

In [22]:
from dataclasses import dataclass

@dataclass
class Point:
    x: float
    y: float

# Creating an instance of Point
point1 = Point(1.5, 2.5)
print(point1)  # Output: Point(x=1.5, y=2.5)

# Accessing fields
print(point1.x)  # Output: 1.5
print(point1.y)  # Output: 2.5

# Comparing instances
point2 = Point(1.5, 2.5)
print(point1 == point2)  # Output: True

# Modifying fields
point1.x = 3.0
print(point1)  # Output: Point(x=3.0, y=2.5)


Point(x=1.5, y=2.5)
1.5
2.5
True
Point(x=3.0, y=2.5)
